In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f


spark = SparkSession.builder.appName('SparkByExamples.com')\
    .config('spark.driver.bindAddress','localhost')\
    .config("spark.ui.port","4051")\
    .config("spark.driver.memory","5g")\
    .getOrCreate()


23/02/17 20:57:04 WARN Utils: Your hostname, Chaturvedi_PC resolves to a loopback address: 127.0.1.1; using 172.28.61.13 instead (on interface eth0)
23/02/17 20:57:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/17 20:57:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# all_train_input_path = '/home/abhay/work/dream11/processed_output/training_rows'
# all_train_df = spark.read.parquet(all_train_input_path)

# output_base_path = '/home/abhay/work/dream11/model_data/model_inputs'
# splits = all_train_df.randomSplit([0.7, 0.3], 42)
# train_df = splits[0]
# test_df = splits[1]

# train_df.write.format("parquet").mode("overwrite").save(output_base_path+"/train")
# train_df.write.format("parquet").mode("overwrite").save(output_base_path+"/test")

In [2]:
train_df = spark.read.parquet("/home/abhay/work/dream11/model_data/model_inputs/train")
test_df = spark.read.parquet("/home/abhay/work/dream11/model_data/model_inputs/test")

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
features = [
     'batter_runs_30D',
     'batter_runs_90D',
     'batter_runs_300D',
     'batter_runs_1000D',
     'batter_runs_1000D_venue',
     'balls_faced_30D',
     'balls_faced_90D',
     'balls_faced_300D',
     'balls_faced_1000D',
     'balls_faced_1000D_venue',
     'dismissals_30D',
     'dismissals_90D',
     'dismissals_300D',
     'dismissals_1000D',
     'dismissals_1000D_venue',
     'boundary_count_30D',
     'boundary_count_90D',
     'boundary_count_300D',
     'boundary_count_1000D',
     'boundary_count_1000D_venue',
     'six_count_30D',
     'six_count_90D',
     'six_count_300D',
     'six_count_1000D',
     'six_count_1000D_venue',
     'batting_avg_30D',
     'batting_avg_90D',
     'batting_avg_300D',
     'batting_avg_1000D',
     'batting_avg_1000D_venue',
     'batting_sr_30D',
     'batting_sr_90D',
     'batting_sr_300D',
     'batting_sr_1000D',
     'batting_sr_1000D_venue',
     'total_runs_30D',
     'total_runs_90D',
     'total_runs_300D',
     'total_runs_1000D',
     'total_runs_1000D_venue',
     'deliveries_30D',
     'deliveries_90D',
     'deliveries_300D',
     'deliveries_1000D',
     'deliveries_1000D_venue',
     'wicket_sum_30D',
     'wicket_sum_90D',
     'wicket_sum_300D',
     'wicket_sum_1000D',
     'wicket_sum_1000D_venue',
     'maiden_count_30D',
     'maiden_count_90D',
     'maiden_count_300D',
     'maiden_count_1000D',
     'maiden_count_1000D_venue',
     'bowling_avg_30D',
     'bowling_avg_90D',
     'bowling_avg_300D',
     'bowling_avg_1000D',
     'bowling_avg_1000D_venue',
     'bowling_sr_30D',
     'bowling_sr_90D',
     'bowling_sr_300D',
     'bowling_sr_1000D',
     'bowling_sr_1000D_venue',
     'bowling_eco_30D',
     'bowling_eco_90D',
     'bowling_eco_300D',
     'bowling_eco_1000D',
     'bowling_eco_1000D_venue'
]

In [16]:
va = VectorAssembler(inputCols = features, outputCol='features')

va_train_df = va.transform(train_df)
va_train_df.show(3)

# (train, test) = va_df.randomSplit([0.8, 0.2])

gbtr = GBTRegressor(featuresCol='features', labelCol='fantasy_points', maxIter=10)
gbtr = gbtr.fit(va_train_df)

+-----------+--------------+-----------+----------+--------------+---------+---------------+---------------+----------------+-----------------+-----------------------+---------------+---------------+----------------+-----------------+-----------------------+--------------+--------------+---------------+----------------+----------------------+------------------+------------------+-------------------+--------------------+--------------------------+-------------+-------------+--------------+---------------+---------------------+---------------+---------------+----------------+-----------------+-----------------------+--------------+--------------+---------------+----------------+----------------------+---------------+-------------+----------+----------+------------+--------------+--------------+---------------+----------------+----------------------+--------------+--------------+---------------+----------------+----------------------+--------------+--------------+---------------+---------

In [10]:
va_test_df = va.transform(test_df)
# mdata = gbtr.transform(va_test_df)
test_predictions = gbtr.transform(va_test_df)
test_predictions.show(3)

+-----------+--------------+-----------+----------+--------------+---------+---------------+---------------+----------------+-----------------+-----------------------+---------------+---------------+----------------+-----------------+-----------------------+--------------+--------------+---------------+----------------+----------------------+------------------+------------------+-------------------+--------------------+--------------------------+-------------+-------------+--------------+---------------+---------------------+---------------+---------------+----------------+-----------------+-----------------------+--------------+--------------+---------------+----------------+----------------------+---------------+-------------+----------+----------+------------+--------------+--------------+---------------+----------------+----------------------+--------------+--------------+---------------+----------------+----------------------+--------------+--------------+---------------+---------

In [9]:
rmse=RegressionEvaluator(labelCol="fantasy_points", predictionCol="prediction", metricName="rmse")
rmse=rmse.evaluate(mdata) 
 
mae=RegressionEvaluator(labelCol="fantasy_points", predictionCol="prediction", metricName="mae")
mae=mae.evaluate(mdata) 
 
r2=RegressionEvaluator(labelCol="fantasy_points", predictionCol="prediction", metricName="r2")
r2=r2.evaluate(mdata)

print("RMSE: ", rmse)
print("MAE: ", mae)
print("R-squared: ", r2)

RMSE:  20.540865073050384
MAE:  13.700677515009605
R-squared:  0.5785430478770521


In [19]:
output_test_path = "/home/abhay/work/dream11/model_data/model_oututs/test"
test_predictions.write.format("parquet").partitionBy(["dt"]).mode("overwrite").save(output_test_path)

In [20]:
train_predictions = gbtr.transform(va_train_df)

In [21]:
output_train_path = "/home/abhay/work/dream11/model_data/model_oututs/train"
train_predictions.write.format("parquet").partitionBy(["dt"]).mode("overwrite").save(output_train_path)